In [7]:
import Pkg
Pkg.add("POMDPModels")
Pkg.add("TabularTDLearning")

using POMDPs
using POMDPModels
using TabularTDLearning
using POMDPPolicies
using RLInterface
using IJulia
using POMDPModelTools
IJulia.clear_output(true)

using DMUStudent.HW4


┌ Info: Precompiling DMUStudent [a176809d-bc77-421e-b60b-7b215d415e6e]
└ @ Base loading.jl:1273


In [5]:
m = SimpleGridWorld();
env = MDPEnvironment(m, GWPos);
const Q = zeros(length(states(m)), length(actions(m)));

In [6]:
function run_episode!(Q; α=0.1, ϵ=0.1, show=false, max_t=200)
    A = actions(m)
    done = false
    rsum = 0.0
    s = reset!(env)
    t = 1
    γ = 0.95
    while t <= max_t
        si = stateindex(m, s)
        
        # select action (eps greedy)
        if rand() < ϵ
            a = rand(A)
        else
            a = A[argmax(Q[si, :])]
        end
        ai = actionindex(m, a)
        
        sp, r, done, info = step!(env, a)
       
        rsum += γ^t*r
        
        spi = stateindex(m, sp)
        Q[si,ai] += α * (r + γ * maximum(Q[spi,:]) - Q[si,ai])
        
        if done
            break
        end
        
        if show
            IJulia.clear_output(true)
            display(POMDPModelTools.render(m, (s=s,),
                color=s->maximum(Q[stateindex(m, s),:]),
                policy=ValuePolicy(m, Q)
            ))
        end
        
        s = sp
        t += 1
    end
    return rsum
end

run_episode! (generic function with 1 method)

In [ ]:
run_episode!(Q, show=true, ϵ=0.9, max_t=20)

In [ ]:
for i in 1:10000
    run_episode!(Q)
end
@show run_episode!(Q)

In [ ]:
display(POMDPModelTools.render(m, NamedTuple(),
                color=s->maximum(Q[stateindex(m, s),:]),
                policy=ValuePolicy(m, Q)
            ))